In [ ]:
from pathlib import Path
TEST_ROOT = Path(__file__).parent.parent / "tests"
TEST_DIR = TEST_ROOT / "test_data"

In [ ]:
from pymatgen.core.structure import Structure
si_structure = Structure.from_file(TEST_DIR / "structures" / "Si.cif")

In [ ]:
from atomate2.forcefields.flows.phonons import PhononMaker
from jobflow import run_locally
flow=PhononMaker(
        min_length=3.0,
        born_maker=None,
        use_symmetrized_structure="conventional",
        create_thermal_displacements=False,
        store_force_constants=False,
        prefer_90_degrees=False,
        generate_frequencies_eigenvectors_kwargs={"tstep": 100},
    ).make(si_structure)
run_locally(flow, store=job_store,create_folders=True, raise_immediately=True)